In [17]:
import mdtraj as md
import pairing
import numpy as np

# We will demonstrate how to calculate ion pairs in a solution of ionic liquids
### Pairing of molecules is determined by a distance criteria, and the information is stored in a direct correlation matrix.  The direct correlation matrix is constructed from a diagonal matrix of size NxN where N is the number of molecules in the system.  Each row (or column) can be thought of the connections that a particular molecule has with the other molecules in the system.  If molecule `i` is more than distance criteria away from molecule `j`, they are not connected and a 0 is placed in the `i,j` position of the matrix.  If the distance between the two molecules are less than the distance criteria, the molecules are considered to be paired and a 1 is placed in the position instead.
### For this example, we will load in a trajectory containing the ionic liquid, ($[BMIM^+][Tf_2N^-]$), and acetonitrile.  We will calculate the pairs and clusters of ions in solution.

### First, let's use MDTraj to load a GROMACS trajectory

In [66]:
trj = md.load('sample_files/slice.xtc', top='sample_files/sample.gro')

### Now we will slice the trajectory to only include the ions $BMIM^+$ and $Tf_2N^-$

In [67]:
sliced = trj.topology.select('resname bmim tf2n')
trj_slice = trj.atom_slice(sliced)

### Next let's call `pairing.calc_direct` to calculate a direct correlation matrix for each frame of the trajectory.  We choose 0.53 nm as our `cutoff` distance as this correlates to the first radial distribution (RDF) peak between $BMIM^+$ and $Tf_2N^-$ 

In [68]:
pairs = pairing.calc_direct(trj_slice, cutoff=0.53)

### Now that we've calculated direct correlation matrices for each frame, let's inspect the matrix for the first frame and see what's going on.

In [69]:
pairs[0]

array([[1, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)

### From first glance, we can see from the 1's in non-diagonal indices that some ions are paired with each other.  Let's take a look at the pairings for the ion in the first row.

In [70]:
first = pairs[0][0]

In [71]:
print(first)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0]


In [72]:
np.argwhere(first>0)

array([[ 0],
       [37]])

### By running `np.argwhere`, we see that there is a value of 1 in two indices of the first row of the matrix: 0 and 37.  We can ignore the 1 in the index 0 as this corresponds to the ion we are computing pairs for.  However, we see that ion 1 is paired to ion 38.

### Now we can loop through all the rows in this matrix and check how many pairs each ions has.

In [73]:
for i,j in enumerate(pairs[6]):
    num_pairs = len(np.argwhere(j>0)) - 1
    print('ion {} has {} total pairs'.format(i+1, num_pairs))

ion 1 has 1 total pairs
ion 2 has 0 total pairs
ion 3 has 0 total pairs
ion 4 has 0 total pairs
ion 5 has 0 total pairs
ion 6 has 0 total pairs
ion 7 has 0 total pairs
ion 8 has 0 total pairs
ion 9 has 0 total pairs
ion 10 has 0 total pairs
ion 11 has 0 total pairs
ion 12 has 0 total pairs
ion 13 has 1 total pairs
ion 14 has 0 total pairs
ion 15 has 0 total pairs
ion 16 has 1 total pairs
ion 17 has 0 total pairs
ion 18 has 0 total pairs
ion 19 has 0 total pairs
ion 20 has 0 total pairs
ion 21 has 0 total pairs
ion 22 has 0 total pairs
ion 23 has 0 total pairs
ion 24 has 1 total pairs
ion 25 has 0 total pairs
ion 26 has 0 total pairs
ion 27 has 1 total pairs
ion 28 has 0 total pairs
ion 29 has 0 total pairs
ion 30 has 0 total pairs
ion 31 has 0 total pairs
ion 32 has 0 total pairs
ion 33 has 0 total pairs
ion 34 has 0 total pairs
ion 35 has 0 total pairs
ion 36 has 0 total pairs
ion 37 has 0 total pairs
ion 38 has 1 total pairs
ion 39 has 0 total pairs
ion 40 has 0 total pairs


### We can also check the total number of pairs in this trajectory frame.  We will do this by counting pairs in each row and then dividing by 2 to double-counting the pairs.

In [74]:
total_pairs = 0
for i,j in enumerate(pairs[0]):
    num_pairs = len(np.argwhere(j>0)) - 1
    total_pairs += num_pairs
total_pairs /= 2
print('The total number of ion pairs in this frame is {}'.format(int(total_pairs)))

The total number of ion pairs in this frame is 4


### We can also compute if there are any clusters of ions in solution.  To do this we first convert the direct correlation matrix to an indirect correlation matrix.  An indirect connection is when ion i is paired with ion j, and ion j is paired with ion k.  In this instance, ion i is indirectly correlated to ion k.

In [77]:
indirect = pairing.calc_indirect(pairs)

### From the indirect correlation matrix, we can determine if any clusters of ions exist in our system.  

In [83]:
clusters = pairing.calc_reduc(indirect)